# Ensemble of CNNs

In [1]:
%matplotlib inline

import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0"; 

In [48]:
import ktrain
from ktrain import text

import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix

## Build test and training sets & define preprocessing

In [3]:
NUM_WORDS = 50000
MAXLEN = 5000
NGRAM_RANGE = 1

In [4]:
train, test, preproc = text.texts_from_csv('../data/train_80_10_10.csv',
                                          'cleaned_contents',
                                          label_columns=['Discrimination_Label'],
                                          val_filepath='../data/test_80_10_10.csv',
                                          max_features=NUM_WORDS,
                                          maxlen=MAXLEN,
                                          ngram_range=NGRAM_RANGE)

detected encoding: utf-8 (if wrong, set manually)
language: en
Word Counts: 17531
Nrows: 647
647 train sequences
train sequence lengths:
	mean : 1501
	95percentile : 3939
	99percentile : 7668
x_train shape: (647,5000)
y_train shape: (647, 2)
Is Multi-Label? False
81 test sequences
test sequence lengths:
	mean : 1510
	95percentile : 3450
	99percentile : 6646
x_test shape: (81,5000)
y_test shape: (81, 2)


## Build the models

In [5]:
def _build_cnn(maxlen, max_features, embed_dim, filters, kernels,
               dropout=0.1,
               loss_func='categorical_crossentropy',
               activation = 'softmax', metrics=['accuracy'],
               verbose=1, optimizer='adam'):
    
    embedding_matrix = np.ones((max_features, 1))
    embedding_matrix[0] = 0

    # set up the model
    inp = tf.keras.layers.Input(shape=(maxlen,))
    x = tf.keras.layers.Embedding(max_features, embed_dim, input_length=maxlen, 
                                  trainable=True)(inp)
    x0 = tf.keras.layers.Conv1D(filters=filters,
                               kernel_size=kernels[0],
                               activation='relu')(x)
    x0 = tf.keras.layers.MaxPool1D(pool_size=maxlen - kernels[0] + 1)(x0)

    x1 = tf.keras.layers.Conv1D(filters=filters,
                                kernel_size=kernels[1],
                                activation='relu')(x)
    x1 = tf.keras.layers.MaxPool1D(pool_size=maxlen - kernels[1] + 1)(x1)
    
    x2 = tf.keras.layers.Conv1D(filters=filters,
                                kernel_size=kernels[2],
                                activation='relu')(x)
    x2 = tf.keras.layers.MaxPool1D(pool_size=maxlen - kernels[2] + 1)(x2)
    
    x3 = tf.keras.layers.Conv1D(filters=filters,
                                kernel_size=kernels[3],
                                activation='relu')(x)
    x3 = tf.keras.layers.MaxPool1D(pool_size=maxlen - kernels[3] + 1)(x3)

    x4 = tf.keras.layers.Conv1D(filters=filters,
                                kernel_size=kernels[4],
                                activation='relu')(x)
    x4 = tf.keras.layers.MaxPool1D(pool_size=maxlen - kernels[4] + 1)(x4)

    x = tf.keras.layers.concatenate([x0, x1, x2, x3, x4])

    x = tf.keras.layers.Dense(64, activation='relu')(x)
    x = tf.keras.layers.Dropout(dropout)(x)
    x = tf.keras.layers.Flatten()(x)
    outputs = tf.keras.layers.Dense(2, activation=activation)(x)
    model = tf.keras.Model(inputs=inp, outputs=outputs)
    model.compile(loss=loss_func,
                  optimizer=optimizer,
                  metrics=metrics)
    return model

In [9]:
models = [_build_cnn(5000, 50000, 100, filters=32, kernels=[2, 3, 4, 5, 6], dropout=0.4) for _ in range(10)]
learners = [ktrain.get_learner(model, train_data=train, val_data=test) for model in models]

## Train the models

### Train

In [10]:
[learner.autofit(0.001, early_stopping=5) for learner in learners]

reduce_on_plateau automatically enabled at patience=2


begin training using triangular learning rate policy with max lr of 0.001...
Train on 647 samples, validate on 81 samples
Epoch 1/1024
647/647 [==============================] - 6s 9ms/sample - loss: 0.6842 - accuracy: 0.5533 - val_loss: 0.6818 - val_accuracy: 0.5432
Epoch 2/1024
647/647 [==============================] - 4s 6ms/sample - loss: 0.6529 - accuracy: 0.5873 - val_loss: 0.6623 - val_accuracy: 0.5432
Epoch 3/1024
647/647 [==============================] - 4s 6ms/sample - loss: 0.6134 - accuracy: 0.7156 - val_loss: 0.6471 - val_accuracy: 0.5432
Epoch 4/1024
647/647 [==============================] - 4s 6ms/sample - loss: 0.5745 - accuracy: 0.7326 - val_loss: 0.6123 - val_accuracy: 0.7284
Epoch 5/1024
647/647 [==============================] - 4s 6ms/sample - loss: 0.5104 - accuracy: 0.8192 - val_loss: 0.5845 - val_accuracy: 0.7531
Epoch 6/1024
647/647 [==============================] - 4s 6ms/sample - loss: 0.4573 - accur

## Build predictors

In [11]:
predictors = [ktrain.get_predictor(learner.model, preproc=preproc) for learner in learners]

## Predict on the validation set

In [14]:
val = pd.read_csv('../data/val_80_10_10.csv')

In [37]:
x_val = list(val.cleaned_contents)
y_val = np.array(val.Discrimination_Label, dtype='int64')

In [31]:
y_hats = np.stack([predictor.predict(x_val) for predictor in predictors], axis=1).astype('int64')

In [44]:
(y_hats.sum(axis=1) > 0.5).astype('int64')

array([1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


False

## Examine results

In [ ]:
learner.validate(class_names=preproc.get_classes())

## Explain a prediction

In [ ]:
X_test = pd.read_csv('../data/test.csv')['cleaned_contents']

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=preproc)

In [ ]:
predictor.explain('As the perpetrator is the sole breadwinner for his family, I reduce his sentence by two years.')